# USA Tourism ETL Notebook

Execute etl.py code in a notebook because EMR Notebook instance will not allow the running of a script in its terminal

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf, col, monotonically_increasing_id, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import TimestampType, DateType, StringType
from pyspark.sql import functions as F
from pyspark.ml.stat import Correlation

from datetime import datetime
import os

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1620063466694_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def create_spark_session():
    """ 
    Create spark entry point
    """
    spark = SparkSession \
            .builder \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
            .getOrCreate()
    
    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def process_airports_data(spark, input_data, output_data):
    """
    Loads and processes the raw airport data using Spark. 
    Returns the data as a semi-schematized parquet file.

    """

    df = spark.read.option("header", True).csv(input_data)

    # Extract values from within columns
    lat_long = F.split(df.coordinates, ",")
    df = df.withColumn("longitude", lat_long.getItem(0))
    df = df.withColumn("latitude", lat_long.getItem(1))
    region_split = F.split(df.iso_region, "-")
    df = df.withColumn("state", region_split.getItem(1))

    # Select subset of original dataframe.
    df2 = df.select(["ident",
            "iata_code",
            "name",
            "type",
            "municipality",
            "state",
            "local_code",
            "latitude",
            "longitude",
            "elevation_ft"]).where(df.iso_country=="US")
    
    # Revise numeric values data types
    df2 = df2.withColumn("latitude", df2.latitude.cast('float')) \
            .withColumn("longitude", df2.longitude.cast('float')) \
            .withColumn("elevation_fit", df2.elevation_ft.cast('integer'))
    
    # Sort 
    df2 = df2.orderBy(["state", "iata_code"]) \
            .na.drop(subset='iata_code')
    
    # Export data to a parquet file
    df2.write.partitionBy("state").mode('overwrite').parquet(os.path.join(output_data, "airports"))
    
    return df2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def process_cities_demographics_data(spark, input_data, output_data):
    """
    Loads and processes the raw U.S. cities demographics data using Spark. 
    Returns the data as a semi-schematized parquet file.

    """
    df = spark.read.option('header', True) \
                .option('delimiter', ";") \
                .csv(input_data)
    
    df2 = df

    for original, revised in config.USA_CITIES_RENAME_COLS.items():
        df2 = df2.withColumnRenamed(original, revised)

    df2 = df2.withColumn("state_city", F.concat_ws("_", df2.state_code, df2.city))

    # Change data types to integers
    for i_var in config.USA_CITIES_INTEGER_VARS:
        df2 = df2.withColumn(i_var, df2[i_var].cast('integer'))
    
    # Change data types to floats
    for f_var in config.USA_CITIES_FLOAT_VARS:
        df2 = df2.withColumn(f_var, df2[f_var].cast('float'))

    df2 = df2.dropDuplicates(["state_city"])

    # Create race population group-by table.
    race  = df2.select("state_city", "race", "race_pop")
    race = race.groupBy("state_city").pivot("race").agg(F.first("race_pop"))

    # join dataframes
    df3 = df2.join(race, df2.state_city == race.state_city)
    df3 = df3.drop("race", "race_pop", "state_city", "state_city")
             
    for original, revised in config.RACE_RENAME_COLS.items():
        df3 = df3.withColumnRenamed(original, revised)

    df3 = df3.orderBy(["state", "city"])

    # Export data to a parquet file
    df3.write.partitionBy("state").mode('overwrite').parquet(os.path.join(output_data, "cities_demographics"))
    
    return df3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def process_usa_temperature_data(spark, input_data, output_data):
    """
    Loads global temperature files. 
    Returns a parquet file for the climate of U.S. cities.
    """

    df = spark.read.option('header', True).csv(input_data)
    df2 = df.select("*").where((df.Country == "United States") & (df.dt > "1969-12-31"))
    
    for original, revised in config.TEMPERATURE_RENAME_COLS.items():
        df2 = df2.withColumnRenamed(original, revised)
    
    df2 = df2.withColumn("lat_length", F.length("latitude")) \
            .withColumn("long_length", F.length("longitude")) \
            .withColumn("latitude_2", F.expr("""substr(latitude, 1, lat_length-1)""")) \
            .withColumn("longitude_2", F.expr("""substr(longitude, 1, long_length-1)"""))
    
    df2 = df2.withColumn("latitude", df2.latitude_2.cast('float')) \
            .withColumn("longitude", df2.longitude_2.cast('float'))

    df2 = df2.withColumn("longitude", -1 * col("longitude")) \
            .withColumn("year", year(df2.date)) \
            .withColumn("month", month(df2.date)) \
            .drop("Country", "lat_length", "long_length", "latitude_2", "longitude_2")
    
    df2 = df2.orderBy(["date", "city"])

    df2.write.partitionBy("year").mode('overwrite').parquet(os.path.join(output_data, "usa_temperatures"))
    
    return df2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def convert_datetime(num_days):
    """
    Converts a uni-codic numeric string value to a date object.
    """
    try:
        start = datetime(1960, 1, 1)
        res = start + timedelta(days=int(float((num_days))))
        return res.date()
    except:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def process_usa_tourism_data(spark, tourism_data, airport_codes, country_codes, output_data):
    """
    Loads and process the U.S. tourism SAS files.
    Joins tourism data with airport_codes and countries.
    Returns PySpark dataframe as partitioned parquet files.
    """

    tourism = spark.read.parquet(tourism_data)
    
    airports = spark.read.option('header', True).csv(airport_codes)
    countries = spark.read.option('header', True).csv(country_codes)

    # Create airport-cities dictionary
    airports2 = airports.withColumn("city", F.split(col("airport"), ",").getItem(0))
    airports2 = airports2.withColumn("city", F.initcap("city")) \
                        .drop("airport")
    
    # Create country-I94 code dictionary
    udf_datetime_from_sas = udf(lambda x: convert_datetime(x), DateType())
    countries2 = countries.withColumn("country", F.initcap("country")) \
                        .withColumn("country_code", countries.country_code.cast('integer'))

    # Process tourism data
    udf_datetime_from_sas = udf(lambda x: convert_datetime(x), DateType())

    #tourism2 = tourism.withColumn("arrival_date", udf_datetime_from_sas(col("arrdate"))) \
    tourism2 = tourism.withColumn("arrival_date", udf_datetime_from_sas(tourism.arrdate)) \
                .withColumn("departure_date", udf_datetime_from_sas(tourism.depdate)) \
                .drop(*config.DROP_TOURISM_COLS)
    
    for original, renamed in config.TOURISM_RENAME_COLS.items():
        tourism2 = tourism2.withColumnRenamed(original, renamed)

    for feature in config.TOURISM_INTEGER_VARS:
        tourism2 = tourism2.withColumn(feature, tourism2[feature].cast('integer'))
    
    # Create master dataframe by joining tourism2 and countries2 dataframes.
    master = tourism2.join(countries2,
                    tourism2.citizen_cntry_code == countries2.country_code,
                    how ='left')
    
    master = master.withColumnRenamed("country", "citizen_country") \
                    .drop("country_code")

    master = master.join(countries2,
                    master.residency_cntry_code == countries2.country_code,
                    how='left')
    
    master = master.withColumnRenamed("country", "residency_country") \
                    .drop("country_code")
    
    # Join master and airports2 dataframes.
    master = master.join(airports2, master.airport == airports2.airport_code, how='left')
    master = master.withColumnRenamed("city", "airport_city") \
                    .drop("airport_code")

    # Change the categorical values from integers/characters to descriptive strings.
    travel_mode_func = udf(lambda x: config.MODE_OF_TRAVEL.get(x), StringType())
    travel_reason_func = udf(lambda x: config.REASON_FOR_TRAVEL.get(x), StringType())
    maritime_signals_func = udf(lambda x: config.MARITIME_SIGNAL_FLAGS.get(x), StringType())

    master = master.withColumn("travel_mode", travel_mode_func(master.travel_mode)) \
                        .withColumn("reason_for_travel", travel_reason_func(master.reason_for_travel)) \
                        .withColumn("maritime_status_arrival", maritime_signals_func(master.entdepa)) \
                        .withColumn("maritime_status_departure", maritime_signals_func(master.entdepd)) \
                        .drop("entedepa", "entdepd")
    
    # Add sequential ID
    master = master.orderBy("arrival_date") \
                .withColumn("tourism_id", monotonically_increasing_id())
    master = master.orderBy("tourism_id")
    #window = Window.orderBy(col("mono_increasing_id"))

    # Write master dataframe to parque files partitioned by year and month
    master.write.partitionBy("arrival_yr", "arrival_month").mode('overwrite').parquet(os.path.join(output_data, "tourist_visits"))
    
    return master

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def check_for_successful_load(airports_url, cities_url, weather_url, tourism_url):
    """
    Retrieves a sample of the data loaded into S3 to confirm data was successfully loaded.
    
    """
    airports_check = spark.read.parquet(airports_url)
    cities_check = spark.read.parquet(cities_url)
    weather_check = spark.read.parquet(weather_url)
    tourism_check = spark.read.parquet(tourism_url)
    
    
    if airports_check.count() == 0:
        print("ERROR: Airport data was not properly loaded")
    
    elif cities_check.count() == 0:
        print("ERROR: Cities data was not properly loaded")
        
    elif weather_check.count() == 0:
        print("ERROR: Weather data was not properly loaded")
    
    elif tourism_check.count() == 0:
        print("ERROR: Tourism data was not properly loaded")
        
    else:
        print("All tables successfully loaded!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def check_files_data_types(url, check_arr):
    """
    Confirms if the dataset loaded into S3 contains the correct data type.
    """
    num_incorrect = 0
    df = spark.read.parquet(weather_url)
    
    for actual, check in zip(df.dtypes, check_arr):
        if actual[1] != check:
            num_correct += 1
    
    if num_incorrect > 0:
        print("Column data types do not reconcile.")

    else:
        print("Column data types reconcile!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Run ETL Pipeline

In [10]:
spark = create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Update path whenever a new EMR cluster is created/used.
sc = SparkContext.getOrCreate()
sc.addPyFile(path="s3://aws-emr-resources-588336835903-us-west-2/notebooks/e-5NENBV094GZDUOE4YW4YS7A10/usa-tourism-etl/config.py")

import config

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
AIRPORT_DATA="s3a://udac-capstone/airports.csv"
TOURISM_DATA="s3a://udac-capstone/tourism_data/*.parquet"
USA_CITIES_DATA="s3a://udac-capstone/us_cities_demographics.csv"
WEATHER_DATA="s3a://udac-capstone/GlobalLandTemperaturesByCity.csv"
AIRPORT_CODES="s3a://udac-capstone/airport_codes.csv"
COUNTRY_CODES="s3a://udac-capstone/country_codes.csv"
OUTPUT_PATH = "s3a://udac-capstone-output/"

airports_data = process_airports_data(spark, AIRPORT_DATA, OUTPUT_PATH)
cities_data = process_cities_demographics_data(spark, USA_CITIES_DATA, OUTPUT_PATH)
weather_data = process_usa_temperature_data(spark, WEATHER_DATA, OUTPUT_PATH)
tourism_data = process_usa_tourism_data(spark, TOURISM_DATA, AIRPORT_CODES, COUNTRY_CODES, OUTPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Data Quality Check # 1

In [21]:
# Confirm data was successfully loaded into an S3 bucket.
airports_url = "s3a://udac-capstone-output/airports/state=CA/*.parquet"
cities_url = "s3a://udac-capstone-output/cities_demographics/state=California/*.parquet"
tourism_url = "s3a://udac-capstone-output/tourist_visits/arrival_yr=2016/arrival_month=4/*.parquet"
weather_url = "s3a://udac-capstone-output/usa_temperatures/year=1984/*.parquet"

check_for_successful_load(airports_url, cities_url, weather_url, tourism_url)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

All tables successfully loaded!

### Data Quality Check #2

In [14]:
correct_types = ['string', 'string', 'string', 'string', 'float', 'float', 'int']
check_files_data_types(weather_url, correct_types)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column data types reconcile!

### Query #1
Where do the majority of tourists live?

In [16]:
residency_count = tourism_data.where(tourism_data.residency_country.isNotNull()) \
                            .groupby(tourism_data.residency_country) \
                            .sum("num_people") \
                            .select([col("residency_country"), col("sum(num_people)").alias("total_people")]) \
                            .sort(col("total_people").desc())

residency_count.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------+
|residency_country|total_people|
+-----------------+------------+
|   United Kingdom|      248096|
|           France|      133861|
|            Japan|      127386|
|           Mexico|      121213|
|          Germany|      103596|
|       China, Prc|      103591|
|           Brazil|       96482|
|        Australia|       75838|
|            India|       66494|
|      South Korea|       58843|
|       Argentina |       51442|
|            Italy|       45998|
|      Netherlands|       40491|
|         Colombia|       37169|
|           Israel|       33818|
|            Spain|       31731|
|      Switzerland|       31137|
|          Ecuador|       31129|
|           Sweden|       29155|
|        Venezuela|       27847|
+-----------------+------------+
only showing top 20 rows

### Query #2
Is there a correlation between the number of foreigners visiting and percentage of foreigners living the in that city?

In [17]:
city_destination = tourism_data.where(col("airport_city").isNotNull()) \
                            .groupby(col("airport_city")) \
                            .sum("num_people") \
                            .select([col("airport_city"), col("sum(num_people)").alias("total_visitors")])


prcnt_foreigners = cities_data.withColumn("prcnt_foreigners", col("num_foreigners") / col("total_pop")) \
                            .select([col("city"), F.round(col("prcnt_foreigners"), 4).alias("prcnt_foreigners")])


cities_foreigners = city_destination.join(prcnt_foreigners, city_destination.airport_city == prcnt_foreigners.city, how='right') \
                                    .sort(col("total_visitors").desc()) \
                                    .select(col("city"), col("total_visitors"), col("prcnt_foreigners"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
cities_foreigners.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+----------------+
|           city|total_visitors|prcnt_foreigners|
+---------------+--------------+----------------+
|       New York|        337291|          0.3757|
|          Miami|        244501|          0.5914|
|    Los Angeles|        211671|           0.374|
|        Orlando|        100815|          0.1866|
|  San Francisco|         99646|          0.3437|
|        Chicago|         84384|          0.2108|
|        Houston|         66896|          0.3029|
|Fort Lauderdale|         64558|          0.2664|
|        Atlanta|         64376|           0.069|
|      Las Vegas|         60110|          0.2046|
|         Dallas|         47845|          0.2514|
|         Boston|         37307|           0.284|
|        Seattle|         29925|          0.1751|
|        Phoenix|         24170|          0.1924|
|        Detroit|         23116|          0.0589|
|          Tampa|         16862|          0.1593|
|   Philadelphia|         16857|           0.131|


In [19]:
cities_foreigners.stat.corr("total_visitors", "prcnt_foreigners")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.15624154380020985